# Importing necessary packages

In [4]:
import pandas as pd
import os
import numpy as np
import requests
import json
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import pearsonr
fileDir = os.path.dirname(os.path.realpath('__file__'))

# Reading the data set

In [5]:
data = open(os.path.join(fileDir, 'sts-dev.csv'))
d1 = data.readlines()

In [6]:
data = open(os.path.join(fileDir, 'sts-test.csv'))
d2 = data.readlines()

In [7]:
data = open(os.path.join(fileDir, 'sts-train.csv'))
d3 = data.readlines()
len(d1) + len(d2) + len(d3)

8628

In [8]:
data = d1 + d2 + d3

# Separating Sentence 1, Sentence 2 and Score

In [9]:
df = []
for i in range(len(data)):
    df.append(data[i].split('\t'))
score = []
sentence1 = []
sentence2 = []
for j in range(len(df)):
    score.append(df[j][4])
    sentence1.append(df[j][5])
    b = df[j][6]
    sentence2.append(b.rstrip("\n"))

# Creating DataFrame with columns Sentence1, Sentence2, Score

In [10]:
dict = list(zip(sentence1, sentence2, score))
df = pd.DataFrame(dict, columns = ['Sentence1', 'Sentence2', 'Score'])

In [13]:
df.head()

,Sentence1,Sentence2,Score
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,5.000
1,A young child is riding a horse.,A child is riding a horse.,4.750
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,5.000
3,A woman is playing the guitar.,A man is playing guitar.,2.400
4,A woman is playing the flute.,A man is playing a flute.,2.750


#  Calling web service using POST method with this request
### Model's
* USE (Universal Sentence Encoder)
* BERT (Bidirectional Encoder Representations from Transformers)

In [252]:
BERT_SimilarityScore = []
USE_SimilarityScore = []
use = []
bert = []

model = ["USE", "BERT"]
for y in model:
    url = 'http://192.168.0.9:5010/sentence_similarity?model='+y
    print(url)
    for i in range(len(df)):
        a = df.Sentence1[i].strip(",")
        b = df.Sentence2[i].strip(",")
        d = { "sentence1":"\'"+str([df.Sentence1[i].replace(",", "")])+"\'", "sentence2":"\'"+str([df.Sentence2[i].replace(",", "")])+"\'"}
        r = requests.post(url, data=d)
        if y == "USE":
            use.append(r.json())
        else:
            bert.append(r.json())
for u in range(len(use)):
    USE_SimilarityScore.append(float(use[u][0][0]["similarityScore"]))
    
for k in range(len(use)):
    BERT_SimilarityScore.append(float(bert[k][0][0]["smilarity_score"]))

http://192.168.0.9:5010/sentence_similarity?model=USE
http://192.168.0.9:5010/sentence_similarity?model=BERT


## Adding 2 new columns to the dataframe (USE_SimilarityScore & BERT_SimilarityScore) and also rescaling the Score column between 0 to 1 

In [253]:
dic = list(zip(sentence1, sentence2, score, USE_SimilarityScore, BERT_SimilarityScore))
df = pd.DataFrame(dic, columns = ['Sentence1', 'Sentence2', 'Score', 'USE_SimilarityScore', 'BERT_SimilarityScore'])

scaler = MinMaxScaler()
df['Score'] = scaler.fit_transform(df['Score'].values.reshape(-1,1))

In [254]:
df.head()

,Sentence1,Sentence2,Score,USE_SimilarityScore,BERT_SimilarityScore
0,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.,1.00,0.992501,0.996232
1,A young child is riding a horse.,A child is riding a horse.,0.95,0.989279,0.993364
2,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.,1.00,0.976037,0.988224
3,A woman is playing the guitar.,A man is playing guitar.,0.48,0.733619,0.989176
4,A woman is playing the flute.,A man is playing a flute.,0.55,0.805151,0.992878


In [255]:
# import matplotlib.pyplot as plt
# # Plot
# x = df["Score"]
# y = df["USE_SimilarityScore"]
# plt.scatter(x, y, alpha=0.5)
# plt.title('Scatter plot pythonspot.com')
# plt.xlabel('Human Generated Score')
# plt.ylabel('USE Similarity Score')
# plt.show()


# Computing the Pearson correlation between USE semantic similarity scores and human judgement Score.

In [256]:
pearson_coef, p_value = stats.pearsonr(df["Score"], df["USE_SimilarityScore"]) 
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value)

Pearson Correlation Coefficient:  0.75241732107532 and a P-value of: 0.0


# Computing the Pearson correlation between BERT semantic similarity scores and human judgement Score.

In [257]:
pearson_coef, p_value = pearsonr(df["Score"], df["BERT_SimilarityScore"])
print("Pearson Correlation Coefficient: ", pearson_coef, "and a P-value of:", p_value)

Pearson Correlation Coefficient:  0.3887031289745391 and a P-value of: 3.34307788298359e-309


In [258]:
df.corr()

,Score,USE_SimilarityScore,BERT_SimilarityScore
Score,1.000000,0.752417,0.388703
USE_SimilarityScore,0.752417,1.000000,0.344512
BERT_SimilarityScore,0.388703,0.344512,1.000000
